# Extact features

The following notebook extracts the features from each input image (patch). 

The extracted features are:

* 10 H-channel energy bins as a vector: [h0, h1, h2, ... h8, h9]
* 10 S-channel energy bins as a vector: [s0, s1, s2, ... s8, s9]
* 10 V-channel energy bins as a vector: [v0, v1, v2, ... v8, v9]
* 10 standardized LBP bins as a vector: [lbp0, lbp1, ... lbp9]

A vector (40x1) is constructed for each image as:
  - vector: [h0, ... , h9, s0, ... , s9, v0, ... , v9, lbp0, ... , lbp9]

The notebook returns a 'xlsx' called ***extract-features.xlsx***.


## Conect with Drive 

Conect with Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Auxiliar functions

It uses two auxiliary functions: 
- getEnergy(): Convert BGR to HSV, and extract energy for HSV chanels. 
- lbphist(): histogram from LBP. 

In [2]:
def getEnergy(img):
    # Convert BGR to HSV and parse HSV
    hsv_img = rgb2hsv(img)
    h, s, v = hsv_img[:, :, 0], hsv_img[:, :, 1], hsv_img[:, :, 2]
    
    # Get height, width and number of pixels of the image
    height = img.shape[0]
    width = img.shape[1]
    pixels = (height-2) * (width-2)
    energy_h = [0] * pixels
    energy_s = [0] * pixels
    energy_v = [0] * pixels
    pixel = 0
        
    #Calculate the energy of each pixel (channel selected by user)
    #print('Getting energy from H channel')
    for j in range(height-2): #external for, height (start in j+1, end in height-2)
        for k in range(width-2): #internal for, width (start in k+1, end in width-2)
            x1 = h[j+1][k]
            x2 = h[j+1][k+2]
            y1 = h[j][k+1]
            y2 = h[j+2][k+1]
            
            energy = sqrt(pow((x1-x2),2) + pow((y1-y2),2))
            energy_h[pixel] = energy
            pixel += 1
            
    pixel = 0               
    #print('Getting energy from S channel')
    for j in range(height-2): #external for, height (start in j+1, end in height-2)
        for k in range(width-2): #internal for, width (start in k+1, end in width-2)
            x1 = s[j+1][k]
            x2 = s[j+1][k+2]
            y1 = s[j][k+1]
            y2 = s[j+2][k+1]
            
            energy = sqrt(pow((x1-x2),2) + pow((y1-y2),2))
            energy_s[pixel] = energy
            pixel += 1
            
    pixel = 0              
    #print('Getting energy from V channel')
    for j in range(height-2): #external for, height (start in j+1, end in height-2)
        for k in range(width-2): #internal for, width (start in k+1, end in width-2)
            x1 = v[j+1][k]
            x2 = v[j+1][k+2]
            y1 = v[j][k+1]
            y2 = v[j+2][k+1]
            
            energy = sqrt(pow((x1-x2),2) + pow((y1-y2),2))
            energy_v[pixel] = energy
            pixel += 1
    
    # get histogram from eHSV
    (histH, _) = np.histogram(energy_h)
    (histS, _) = np.histogram(energy_s)
    (histV, _) = np.histogram(energy_v)
    histHSV = np.hstack((histH,histS,histV))

    return histHSV #Array of 1 x 30

def lbphist(img,n_points, radius, METHOD):
  n_points = n_points - 2
  lbp = local_binary_pattern(img, n_points, radius, METHOD)
  # get histogram from LBP
  (hist, _) = np.histogram(lbp.ravel(),bins=np.arange(0, n_points + 3),range=(0, n_points + 2))
  # normalize the histogram
  hist = hist.astype("float")
  hist /= (hist.sum() + 1e-7)   
#  print(hist.shape)
  return hist

In [4]:
#path_data = '/content/drive/My Drive/0 - Data/Kindey stones/Test/test-patches-200' #test
path_data = '/content/drive/My Drive/3 - Experiments/6 - Kindey stones/Proyecto_CalculosRenales/Imagenes/Patches/200' #complete

#path_data = 
#print(path_data)
list_folder = ( # Comment on the items on the list that you do NOT want to process
    'au_sections_patches', #test 
    'au_surfaces_patches', #test
    'wd_sections_patches', 
    'wd_surfaces_patches', 
    'ww_sections_patches',
    'ww_surfaces_patches',
    'br_sections_patches',
    'br_surfaces_patches'
    )

## Main function

Three parameters are used for LBP:

- Radius = 3
- Number of pointss (n_points = 10);
- Method = 'default'

In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import pow, sqrt
import skimage
from skimage.color import rgb2hsv 
from skimage import io 
from skimage.feature import local_binary_pattern

# settings for LBP
radius = 3
n_points = 10 # Return 10 values
METHOD = 'default'

cont = 0

for folder in list_folder:
  path_folder = path_data + '/' + folder
  list_images = os.listdir(path_folder)  
  for image in list_images:
    if image != '.ipynb_checkpoints':
      #print(image)
      path_image = path_folder + '/' + image
      imgC = io.imread(path_image, as_gray=False)
      imgBN = io.imread(path_image, as_gray=True)
      energy_HSV = getEnergy(imgC)
      hist = lbphist(imgBN,n_points, radius, METHOD)
      class_ = folder[0:2]
      set_ = folder[3:11]
      features = np.hstack((class_, set_, energy_HSV,hist))
      #print(features)

      if cont == 0:
        list_features = features
        cont += 1
      else:
        list_features = np.vstack((list_features,features))

  print(folder)

df = pd.DataFrame(list_features, columns = ['class', 'set', 'h0','h1','h2','h3','h4','h5','h6','h7','h8','h9','s0','s1','s2','s3','s4','s5','s6','s7','s8','s9','v0','v1','v2','v3','v4','v5','v6','v7','v8','v9','lbp0','lbp1','lbp2','lbp3','lbp4','lbp5','lbp6','lbp7','lbp8','lbp9'])

dats = 2
df_labels = df.iloc[:, 0:dats]
df_h = df.iloc[:, dats+0:dats+10]
df_s = df.iloc[:, dats+10:dats+20]
df_v = df.iloc[:, dats+20:dats+30]
df_lbp = df.iloc[:, dats+30:dats+40]


with pd.ExcelWriter('extract-features.xlsx') as writer:

  df.to_excel(writer, sheet_name='labels-H-S-V-LBP',index = False)
  df_labels.to_excel(writer, sheet_name='labels',index = False)
  df_h.to_excel(writer, sheet_name='H', index = False)
  df_s.to_excel(writer, sheet_name='S', index = False)
  df_v.to_excel(writer, sheet_name='V', index = False)
  df_lbp.to_excel(writer, sheet_name='LBP', index = False)

au_sections_patches
au_surfaces_patches
wd_sections_patches
wd_surfaces_patches
ww_sections_patches
ww_surfaces_patches
br_sections_patches
br_surfaces_patches
